**Table of contents**<a id='toc0_'></a>    
- [Data Loading and Cleaning](#toc1_)    
  - [We only care about Lake Trouts / Seesaiblings!](#toc1_1_)    
- [Preliminary Data Analysis](#toc2_)    
  - [Comparing lakes on Laketrout fishing](#toc2_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Data Loading and Cleaning](#toc0_)

In [42]:
import pandas as pd
import plotly.express as px
import dataLoadingHelper as dlh 

In [43]:
sheets_dict_angling = pd.read_excel('data/angling.xlsx', sheet_name=None)  # Reads all sheets
cleaned_sheets_angling = dlh.makeCleanedSheetDictionary(sheets_dict_angling)

sheets_dict_commercial = pd.read_excel('data/commercialFishing.xlsx', sheet_name=None)  # Reads all sheets
cleaned_sheets_commercial = dlh.makeCleanedSheetDictionary(sheets_dict_commercial)

## <a id='toc1_1_'></a>[We only care about Lake Trouts / Seesaiblings!](#toc0_)

In [44]:
df_trouts_angling = dlh.createTroutFrame(cleaned_sheets_angling, [2000, 2021]) 
df_trouts_commercial = dlh.createTroutFrame(cleaned_sheets_commercial, [2000, 2021]) 
df_trouts_angling = df_trouts_angling.drop(index=0) # removes the Total
df_trouts_angling.head()

/Users/jakobbehler/Documents/EPFL/MA2/Dataviz/project/Dataviz-project-fish/dataLoadingHelper.py:41: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/Users/jakobbehler/Documents/EPFL/MA2/Dataviz/project/Dataviz-project-fish/dataLoadingHelper.py:41: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,Lake,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,Genfersee,3831,2521,5046,4417,2155,3727,2692,1682,3006,...,3903,4031,2971.00,5695,3425,2148,1716.00,2517.0,3464.47,1904
2,Neuenburgersee,1889,1770,1225,1804,512,2375,3006,1587,2424,...,2204,1382,1691.00,2191,1526,908,509.00,543.0,852.00,707
3,Bodensee 2,328,375,477,616,1168,757,419,604,774,...,935,682,447.00,538,438,296,525.00,609.3,670.00,482
4,Vierwaldstättersee,360,368,244,683,726,1550,1072,871,1195,...,1177,1400,1256.00,1497,1475,1793,2219.00,1926.0,948.00,1410
5,Zürichsee 3,242,901,1014,1376,1499,1507,1040,610,454,...,374,380,824.12,1049,785,377,401.42,1723.0,617.00,533


# <a id='toc2_'></a>[Preliminary Data Analysis](#toc0_)

## Stats per Lake
- mean, median, srd and so on.

In [45]:
# TODO

## Plotting the development of trout catches

In [46]:
df_melted = df_trouts_angling.melt(id_vars=["Lake"], var_name="Year", value_name="Fish Catches")
df_melted["Year"] = df_melted["Year"].astype(int)

In [47]:
fig = px.line(df_melted, x="Year", y="Fish Catches", color="Lake",
              title="Lake Trout Catches Over Time for All Lakes",
              labels={"Year": "Year", "Fish Catches": "Fish Catches (Count)", "Lake": "Lake"},
              markers=True)

fig.show()

### Best 5 lakes

In [50]:
# MEAN CATCHES
df_trouts_angling["Mean Catches"] = df_trouts_angling.iloc[:, 1:].mean(axis=1)

# Select top 5 lakes
top_5_lakes = df_trouts_angling.nlargest(5, "Mean Catches")

top_5_lakes = top_5_lakes.drop(columns=["Mean Catches"])
df_melted = top_5_lakes.melt(id_vars=["Lake"], var_name="Year", value_name="Fish Catches")
df_melted["Year"] = df_melted["Year"].astype(int)

In [51]:
# PLOT
fig = px.line(df_melted, x="Year", y="Fish Catches", color="Lake",
              title="Top 5 Lakes with Highest Lake Trout Catches Over Time",
              labels={"Year": "Year", "Lake Trout Catches": "Fish Catches (Count)", "Lake": "Lake"},
              markers=True)

fig.show()

# Mapping the data

In [49]:
import folium
import geopandas as gpd


lakes_data = {
    "Genfersee": {"lat": 46.4500, "lon": 6.5833, "fish_count": 63845},
    "Neuenburgersee": {"lat": 46.925, "lon": 6.85, "fish_count": 23502},
    "Bodensee": {"lat": 47.5, "lon": 9.3, "fish_count": 172600}
}


swiss_map = folium.Map(location=[46.8, 8.3], zoom_start=7)

# Add bubbles for each lake
for lake, data in lakes_data.items():
    folium.CircleMarker(
        location=[data["lat"], data["lon"]],
        radius=data["fish_count"] / 10000, 
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=f"{lake}: {data['fish_count']} fish"
    ).add_to(swiss_map)


swiss_map.save("swiss_lakes_map.html")

